# ***IMPORT DES DONNEES DE COMPTAGES***
> il y a un exemple de verification si doublons de point dans le 17, 47, 87

In [106]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import pandas as pd
import geopandas as gp
import numpy as np
import csv, re, os, statistics, filecmp
import Connexion_Transfert as ct
import Import_trafics as it
import Outils
from collections import Counter
from shapely.geometry import LineString, MultiLineString
from geoalchemy2 import Geometry, WKTElement
from Base_BdTopo import Import_outils as io
from Base_BdTopo import Rond_points as rp
from Base_BdTopo import Regroupement_correspondance as rc
from sqlalchemy.schema import MetaData
from sqlalchemy import inspect
from statistics import mean
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 50)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **CD 23**
- Année 2018 : 
> à partir du fichier __Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD23\2018_CD23_trafics.xls__<br>***attention : pour le point de comptage D941 6+152 à Aubusson, le pR est 32 et non 6. il faut donc corriger à la main le fihcier excel***
<br> Pour le moment tous les points sont déjà dans  la base, dc pas de traitement de type insert prévus.

In [11]:
# ouvrir le classeur
df_excel=pd.read_excel(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD23\2018_CD23_trafics.xls',skiprows=11)
# renomer les champs
df_excel_rennome=df_excel.rename(columns={'1er trimestre  du 01 janvier au 31 mars':'trim1_TV', 'Unnamed: 9':'trim1_pcpl',
                         '2ème trimestre du 01 avril au 30 juin':'trim2_TV', 'Unnamed: 11':'trim2_pcpl',
                         '3ème trimestre du 01 juillet au 30 septembre':'trim3_TV', 'Unnamed: 13':'trim3_pcpl',
                         '4ème trimestre du 01 octobre au 31 décembre':'trim4_TV', 'Unnamed: 15':'trim4_pcpl',
                         'Unnamed: 17':'pc_pl', 'TMJA 2018':'tmja'})
#supprimer la 1ere ligne
df_excel_filtre=df_excel_rennome.loc[1:,:].copy()
#mise en forme attribut
df_excel_filtre['Route']=df_excel_filtre.apply(lambda x : str(x['Route']).upper(), axis=1)
annee_cpt='2018'
#attribut id_comptag
for i in ['DEP','PR','ABS'] : 
    df_excel_filtre[i]=df_excel_filtre.apply(lambda x : str(int(x[i])),axis=1)
df_excel_filtre['id_comptag']=df_excel_filtre.apply(lambda x : '-'.join([x['DEP'],'D'+str(x['Route']),
                                                                         x['PR']+'+'+x['ABS']]),axis=1)
#donnees_mensuelles
list_id_comptag=[val for val in df_excel_filtre.id_comptag.tolist() for _ in (0, 1)]
donnees_type=['tmja','pc_pl']*len(df_excel_filtre.id_comptag.tolist())
annee_df=['2018']*2*len(df_excel_filtre.id_comptag.tolist())
janv, fev, mars,avril,mai,juin,juil,aout,sept,octo,nov,dec=[],[],[],[],[],[],[],[],[],[],[],[]
for i in range(len(df_excel_filtre.id_comptag.tolist())) :
    for j in (janv, fev, mars) :
        j.extend([df_excel_filtre.trim1_TV.tolist()[i],df_excel_filtre.trim1_pcpl.tolist()[i]])
    for k in (avril,mai,juin) :
        k.extend([df_excel_filtre.trim2_TV.tolist()[i],df_excel_filtre.trim2_pcpl.tolist()[i]])
    for l in (juil,aout,sept) :
        l.extend([df_excel_filtre.trim3_TV.tolist()[i],df_excel_filtre.trim3_pcpl.tolist()[i]])
    for m in (octo,nov,dec) :
        m.extend([df_excel_filtre.trim4_TV.tolist()[i],df_excel_filtre.trim4_pcpl.tolist()[i]])
donnees_mens=pd.DataFrame({'id_comptag':list_id_comptag,'donnees_type':donnees_type,'annee':annee_df,'janv':janv,'fevr':fev,'mars':mars,'avri':avril,
              'mai':mai,'juin':juin,'juil':juil,'aout':aout,'sept':sept,'octo':octo,'nove':nov,'dece':dec})

In [12]:
#Mise à jour bdd
with ct.ConnexionBdd('gti_otv') as c :
    c.curs.execute("select distinct id_comptag from comptage.na_2010_2018_p where dep='23' order by id_comptag")
    listerecord=[record[0] for record in c.curs]
    for id_comptag,tmja, pc_pl  in zip(df_excel_filtre.id_comptag.tolist(), df_excel_filtre.tmja.tolist(),df_excel_filtre.pc_pl.tolist()) : 
        if id_comptag in listerecord :
            c.curs.execute("update comptage.na_2010_2018_p set tmja_2018=%s, pc_pl_2018=%s, ann_cpt=%s where id_comptag=%s",(tmja, pc_pl,annee_cpt,id_comptag))
        else : 
            print (f'{id_comptag} nouveau, à traiter')
    print('fini')
    c.connexionPsy.commit()
    donnees_mens.to_sql('na_2010_2018_mensuel', c.sqlAlchemyConn,schema='comptage',if_exists='append',index=False)

23-D941-77+25 ok, MaJ
23-D941-60+270 ok, MaJ
23-D941-57+639 ok, MaJ
23-D941-24+257 ok, MaJ
23-D914-0+600 ok, MaJ
23-D940-39+944 ok, MaJ
23-D997-24+86 ok, MaJ
23-D915-5+595 ok, MaJ
23-D951-8+150 ok, MaJ
23-D990-59+690 ok, MaJ
23-D942-1+585 ok, MaJ
23-D942-23+698 ok, MaJ
23-D1-14+725 ok, MaJ
23-D1-22+760 ok, MaJ
23-D912-15+739 ok, MaJ
23-D5-35+590 ok, MaJ
23-D913-9+780 ok, MaJ
23-D951-28+425 ok, MaJ
23-D6-20+56 ok, MaJ
23-D6-0+826 ok, MaJ
23-D940-50+116 ok, MaJ
23-D940-66+431 ok, MaJ
23-D990-16+988 ok, MaJ
23-D942-31+637 ok, MaJ
23-D990-27+817 ok, MaJ
23-D990-45+295 ok, MaJ
23-D11-6+857 ok, MaJ
23-D11-22+776 ok, MaJ
23-D917-7+372 ok, MaJ
23-D917-12+491 ok, MaJ
23-D993-4+638 ok, MaJ
23-D915-18+907 ok, MaJ
23-D997-35+417 ok, MaJ
23-D996-21+935 ok, MaJ
23-D4-82+80 ok, MaJ
23-D990-65+576 ok, MaJ
23-D941A-4+172 ok, MaJ
23-D982-1+210 ok, MaJ
23-D997-13+862 ok, MaJ
23-D982-17+780 ok, MaJ
23-D982-36+433 ok, MaJ
23-D8-33+506 ok, MaJ
23-D912-42+22 ok, MaJ
23-D4-19+290 ok, MaJ
23-D4-35+917 ok, MaJ


## **CD 40**
> Année 2018 : contient des données mensuelless

In [ ]:
it.cd40()

## **CD17**

> ### Année 2015 : traitée dans le fichier Import_trafics.py

> ### année 2016 données issue des borchures de comptage, uniquemnet pour ponctuels

In [6]:
#ouverture du fichier
cpt17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_produites\donnnees_travail\Comptage\17\cpt_pctuel_but_2016.txt',
                       'brochure',2016)

#mettre à jour les id_comptage deja presents (attention, update Bdd à changé, il faut passer par creer_valeur_txt_update avant et certains paramètres ont changé)
cpt17.mises_forme_bdd('gti_otv', 'comptage', 'na_2010_2017_p', '17','ponctuel') #creer les attributs selon les donnees presentes dans la base
cpt17.update_bdd('gti_otv', 'comptage', 'na_2010_2017_p')#mise à jour

#creer referentiel si besoin
with ct.ConnexionBdd('gti_otv') as c:
    ct.ogr2ogr_shp2pg(c.connstringOgr, r'Y:\REF_GEO\BD_Topo\D17\ED16\SHP\1_DONNEES_LIVRAISON\N_TRONCON_ROUTE_BDT_017.SHP',
                     schema='referentiel',table='troncon_route_bdt17_ed16_l',geotype='MULTILINESTRING', dims=2, encodageClient='LATIN1' )
#creer graph
rqt="""
alter table referentiel.troncon_route_bdt17_ed16_l add column source integer , add column target integer ;
select pgr_createTopology ('referentiel.troncon_route_bdt17_ed16_l',1,'geom', 'ogc_fid') ;
ALTER TABLE referentiel.troncon_route_bdt17_ed16_l  RENAME COLUMN id TO id_ign;
ALTER TABLE referentiel.troncon_route_bdt17_ed16_l  RENAME COLUMN ogc_fid TO id;
alter table referentiel.troncon_route_bdt17_ed16_l add column long_km numeric ;
update referentiel.troncon_route_bdt17_ed16_l set long_km=(st_length(geom)/1000) ;
""" #attention, il manque la ligne au dessus pour créer l'analyseGraph qui va renvoyer le nb de count
with ct.ConnexionBdd('gti_otv') as c:
    c.sqlAlchemyConn.execute(rqt)

#inserer les nouveaux comptages
with ct.ConnexionBdd('gti_otv') as c:
    cpt17.df_attr_insert.to_sql('na_2010_2017_p',c.sqlAlchemyConn,schema='comptage',if_exists='append', index=False )

#mettre à jour la geom 
rqt=""" update comptage.na_2010_2017_p
  set geom=(select geom_out  from comptage.geoloc_pt_comptag(id_comptag))
  where dep='17' and geom is null
"""
with ct.ConnexionBdd('gti_otv') as c:
    c.sqlAlchemyConn.execute(rqt)

> ### fichier compteurs permanents format csv annee 2017 ou 2018

In [201]:
cpt_perm=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\2018_csv_permanents_CD17.csv',
                         'permanent_csv',2018)

cpt_perm.mises_forme_bdd('gti_otv', 'comptage', 'na_2010_2018_p', '17','permanent')

#miettre à jour les données deja existantes
cpt_perm.update_bdd('gti_otv', 'comptage', 'na_2010_2018_p')#mise à jour

#inseérer les données nouvelles
cpt_perm.insert_bdd('gti_otv', 'comptage', 'na_2010_2018_p')
#mettre à jour la geom 
cpt_perm.maj_geom('gti_otv', 'comptage', 'na_2010_2018_p', '17')

# pour les données mensuelles
cpt_perm.insert_bdd_mens('gti_otv', 'comptage','na_2010_2018_mensuel')

> ### annee 2018 ; fichier compteurs tournant format excel issu des donnees pour brochure

In [290]:
#initiliser la classe avec le fichier
bdd='gti_otv_pg11'
cpt_cd17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\Brochure 2018 CD17 DREAL\10  5 1 B3 tournants recalculés.xls',
                      'tournant_xls_bochure',2018)

#mise en forme des données
cpt_cd17.comptag_existant_bdd(bdd, 'na_2010_2018_p', dep='17')
donnees=cpt_cd17.ouvrir_xls_tournant_brochure()
cpt_cd17.conversion_id_comptg_existant_xls_brochure(bdd)
cpt_cd17.carac_xls_brochure()

#mise à jour des données
val_txt=cpt_cd17.creer_valeur_txt_update(cpt_cd17.df_attr_update, ['id_comptag','tmja_2018','tmja_2017'])
cpt_cd17.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja_2018','tmja_2017':'tmja_2017'})

### annee 2017-2018 ; fichier ponctuel excel qui alimente des brochures

In [453]:
#ouvrir le fichier et initialisation
cpt_pct2018_cd17=it.Comptage_cd17(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD17\Brochure 2018 CD17 DREAL\spécifique Brochure V85_2018 .xls',
                      'ponctuel_xls_bochure',2018)
bdd='gti_otv_pg11'
#mise en forme
cpt_pct2018_cd17.comptag_existant_bdd(bdd, 'na_2010_2018_p', dep='17')
cpt_pct2018_cd17.conversion_id_comptg_existant_xls_brochure(bdd)
cpt_pct2018_cd17.filtrer_periode_ponctuels_xls_brochure()
cpt_pct2018_cd17.carac_xls_brochure()

#mise à jour des données
val_txt=cpt_pct2018_cd17.creer_valeur_txt_update(cpt_pct2018_cd17.df_attr_update, ['id_comptag','tmja','pc_pl','obs'])
cpt_pct2018_cd17.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja','pc_pl_2018':'pc_pl', 'obs_2018':'obs'})

In [467]:
#trouver les points de comptages a inserer situe sur le même troncon elementaires que d'autres points
table_corresp=cpt_pct2018_cd17.correspondance_ancien_nouveau_comptage(bdd,'public','cd17_tournant_insert','lineaire.traf2016_bdt17_ed16_l',
                                                    'public', 'traf2016_bdt17_ed16_l','traf2016_bdt17_ed16_l_vertices_pgr')

#si le point fait partie de la table_corresp, on insère pas, sinon on insère
pt_a_inserer=cpt_pct2018_cd17.df_attr_insert.loc[~cpt_pct2018_cd17.df_attr_insert.id_comptag.isin(table_corresp.id_comptag.tolist())].copy()
#mise en form avant insertion
pt_a_inserer['dep']='17'
pt_a_inserer['route']=pt_a_inserer.id_comptag.apply(lambda x : x.split('-')[1])
pt_a_inserer.rename(columns={'absc':'abs','tmja':'tmja_2018','pc_pl':'pc_pl_2018','obs':'obs_2018'},inplace=True)
pt_a_inserer['reseau']='RD'
pt_a_inserer['gestionnai']='CD17'
pt_a_inserer['concession']='N'
pt_a_inserer['type_poste']='ponctuel'
pt_a_inserer=pt_a_inserer[['id_comptag','dep','route','pr','abs','reseau','gestionnai','concession','type_poste','tmja_2018','pc_pl_2018','obs_2018']].copy()
#si plusieurs fois le mm point on garde la valeur max
pt_a_inserer=pt_a_inserer.loc[pt_a_inserer.tmja_2018==pt_a_inserer.groupby('id_comptag').tmja_2018.transform(max)].copy()

cpt_pct2018_cd17.insert_bdd(bdd, 'comptage', 'na_2010_2018_p', pt_a_inserer)
cpt_pct2018_cd17.maj_geom(bdd, 'comptage', 'na_2010_2018_p', dep='17')

## **CD19**
> Annee des trafics 2018, fichier : Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD19\2018_Recensement_trafic.xls

In [60]:
#importer fichier
fichier=r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD19\2018_Recensement_trafic.xls'
donnees_brutes=pd.read_excel(fichier, skiprows=6)
donnees_filtrees=donnees_brutes.rename(columns={'N° R.D.':'route','P.R.':'pr',2018:'ann_2018'})[['route','pr','ann_2018']]
donnees_filtrees=donnees_filtrees.loc[~donnees_filtrees.pr.isna()].copy()

In [61]:
#mettre à jour les champs et pereparer les donnees
def id_comptage(route,pr) : 
    route=str(route).strip()
    pr=str(int(pr.split('+')[0]))+'+0' if int(pr.split('+')[1])==0 else str(int(pr.split('+')[0]))+'+'+str(int(pr.split('+')[1]))
    return '19-D'+route+'-'+pr

donnees_filtrees['idcomptag']=donnees_filtrees.apply(lambda x : id_comptage(x['route'],x['pr']), axis=1)
donnees_filtrees['tmja']=donnees_filtrees.ann_2018.apply(lambda x : 0 if (pd.isna(x) or x=='x') else int(x.split('\n')[0]))
donnees_filtrees['pc_pl']=donnees_filtrees.ann_2018.apply(lambda x : 0 if (pd.isna(x) or x=='x') else float(x.split('\n')[1].split('%')[0].replace(',','.')))
donnees_transfert=donnees_filtrees.loc[donnees_filtrees['tmja']>0].copy()

In [68]:
#pour interactions avec Bdd
bdd='gti_otv_pg11'

In [62]:
#prise en compte variation id_comptag
rqt_corresp_comptg='select * from comptage.corresp_id_comptag'
with ct.ConnexionBdd(bdd) as c:
    corresp_comptg=pd.read_sql(rqt_corresp_comptg, c.sqlAlchemyConn)
donnees_transfert['idcomptag']=donnees_transfert.apply(lambda x : corresp_comptg.loc[corresp_comptg['id_gest']==x['idcomptag']].id_gti.values[0] 
                                            if x['idcomptag'] in corresp_comptg.id_gest.tolist() else x['idcomptag'], axis=1)

In [64]:
#Recherche des points existants
comptage=it.Comptage(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD19\2018_Recensement_trafic.xls')
cpt_existant=comptage.comptag_existant_bdd('gti_otv_pg11','na_2010_2018_p',dep='19')
#identification des nouveaux points
points_a_inserer=donnees_transfert.loc[~donnees_transfert['idcomptag'].isin(cpt_existant.id_comptag.tolist())].copy()
#identification des points à mettre a jour
points_a_mettre_a_jour=donnees_transfert.loc[donnees_transfert['idcomptag'].isin(cpt_existant.id_comptag.tolist())]

In [69]:
#mettre a jour
valeurs_txt=str(tuple([(elem[0],elem[1], elem[2]) for elem in zip(
           points_a_mettre_a_jour.idcomptag.tolist(), points_a_mettre_a_jour.tmja.tolist(), 
            points_a_mettre_a_jour.pc_pl.tolist(), )]))[1:-1]
rqt=f"""update comptage.na_2010_2018_p  as c 
                set tmja_2018=v.tmja,pc_pl_2018=v.pc_pl from (values {valeurs_txt}) as v(id_comptag,tmja,pc_pl)
                where v.id_comptag=c.id_comptag"""
with ct.ConnexionBdd(bdd) as c:
    c.sqlAlchemyConn.execute(rqt)

In [70]:
#inserer
#mise en forme
points_a_inserer.rename(columns={'idcomptag':'id_comptag','tmja':'tmja_2018','pc_pl':'pc_pl_2018'}, inplace=True)
points_a_inserer.drop(['route','pr','ann_2018'], axis=1, inplace=True)
points_a_inserer['type_poste']='tournant'
points_a_inserer['dep']='19'
points_a_inserer['reseau']='RD'
points_a_inserer['gestionnai']='CD19'
points_a_inserer['concession']='N'
with ct.ConnexionBdd(bdd) as c:
    points_a_inserer.to_sql('na_2010_2018_p',c.sqlAlchemyConn,schema='comptage',if_exists='append', index=False )

In [ ]:
comptage.maj_geom(bdd,'comptage','na_2010_2018_p','19')

## **CD47**
> Annee des trafics 2018, fichiers : Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD47\comptages_CD47_2018\COMPTAGES 2018
les fichiers sont decomposes en permanents tournants temporaires, il faut recomposer les donnees

In [195]:
#initialiser un objet
cpt47=it.Comptage_cd47(r'Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD47\comptages_CD47_2018\COMPTAGES 2018','TRAFICS PERMANENTS')
#calculer les attributs de comptages : df_attr, df_attr_insert et df_attr_update, en prenant en compte les comptages existants
cpt47.classer_comptage_update_insert('gti_otv_pg11')

In [177]:
#mise à jour des données déjà présentes dansla base
val_txt=cpt47.creer_valeur_txt_update(cpt47.df_attr_update, ['id_comptag','tmja','pc_pl'])
cpt47.update_bdd(bdd, 'comptage', 'na_2010_2018_p', val_txt,{'tmja_2018':'tmja','pc_pl_2018':'pc_pl'})

In [ ]:
#traitement des données non présentes (df_attr_insert)
#recherche de correspondance pour les permanents et tournants
dico_corresp=cpt47.corresp_old_new_comptag('gti_otv_pg11', 'public','cpt47_temp','lineaire.traf2017_bdt47_ed17_l',
                            'referentiel', 'troncon_route_bdt47_ed17_l','troncon_route_bdt47_ed17_l_vertices_pgr','id')

In [222]:
#filtre des données df_attr_insert selon l'id_comptag present dans le dico_corresp
cpt47.df_attr_insert=cpt47.df_attr_insert.loc[~cpt47.df_attr_insert.id_comptag.isin(df_correspondance.id_comptag.to_list())].copy()
# POUR INFO, l'id_comptag présent dan sle dico_corresp a été transférer à la main dans l table comptage.corresp_id_comptag
#mettre en forme les attributs avant insert
cpt47.mise_en_forme_insert('2018')
#inserer les donnes dans la table
cpt47.insert_bdd('gti_otv_pg11', 'comptage', 'na_2010_2018_p', cpt47.df_attr_insert)

In [227]:
#mettre à jour la géométrie
cpt47.maj_geom('gti_otv_pg11', 'comptage', 'na_2010_2018_p', '47')

## **CD87**
> Dans ce Departement les donnees sont fournies en fichiers .fim. il vaut recalculer les valeusr de comptages <br> Q:\DAIT\TI\DREAL33\2019\C19SA0035_OTR-NA\Doc_travail\Donnees_source\CD87\Fichiers FIM 87 -2018

### ***lister les voies, classer les types de commptages, lister les fichiers a regrouper***
> le dossier contient un tres grand nombre de fihciers, parfois ils sont a regrouper, parfois la structure de nommage varie

In [2]:
d87=it.Comptage_cd87(r'D:\temp\otv\87\Fichiers FIM 87 -2018') 
d87.dico_pt_cptg() #creer le dico de base

OK : tous fichiers dans dico


In [22]:
d87.dico_voie

{'D940': [{'pr': 7,
   'abs': 90,
   'fichiers': ['D 940 PR 07090 2018 01.FIM',
    'RD940_PR07(090_1803011024_M4H_WA.FIM',
    'RD940_PR07090_1809031546_M4H_WA.FIM',
    'RD940_PR07090_2018 03.FIM',
    'RD940_PR07090_2018 04.FIM',
    'RD940_PR07090_2018 05.FIM',
    'RD940_PR07090_2018 06.FIM',
    'RD940_PR07090_2018 09.FIM',
    'RD940_PR07090_2018 10.FIM',
    'RD940_PR07090_2018 11.FIM'],
   'tmja': 1323,
   'pc_pl': 10.11,
   'date_debut': nan,
   'date_fin': nan},
  {'pr': 22,
   'abs': 900,
   'fichiers': ['D940PR22900_2018 09.FIM', '940 22900 2018 02.FIM'],
   'tmja': 385,
   'pc_pl': 10.5,
   'date_debut': nan,
   'date_fin': nan},
  {'pr': 15,
   'abs': 550,
   'fichiers': ['940_15550_2018 02.FIM'],
   'tmja': 1062,
   'pc_pl': 6.8,
   'date_debut': Timestamp('2018-02-21 10:00:00'),
   'date_fin': Timestamp('2018-03-01 13:00:00')}],
 'D05A': [{'pr': 1,
   'abs': 470,
   'fichiers': ['D05A4PR01470_2018 10.FIM'],
   'tmja': 363,
   'pc_pl': 4.7,
   'date_debut': Timestamp('2

### ***Traiter les donnees***
> Pour chaque route référencée dans le dico, on va calculer les TMJA %PL, date_debut, date_fin, type_poste de chaque fichiers puis
faire les calculs si necessaires (moyenne si plsr fichiers).<br> ensuite on classe puis update et insert

In [ ]:
#mettre à jour le dico et le transformer en dataframe sans les ponctuels pendant les grandes vacances
d87.dataframe_dico_glob()
#preparer les données
d87.classer_comptage_update_insert('gti_otv_pg11','na_2010_2018_p','comptage'
                                   'public','d87_cpt_temp','lineaire.traf2017_bdt87_ed17_l',
                                  'referentiel', 'troncon_route_bdt87_ed17_l','troncon_route_bdt87_ed17_l_vertices_pgr','id')
#mettre à jour les données
d87.update_bdd_d87('gti_otv_pg11','na_2010_2018_p','comptage')
#insérer les données et mettre à jour les geom
d87.insert_bdd_d87('gti_otv_pg11','na_2010_2018_p','comptage')

## **CD 16**
- Année 2018 : 
> à partir du fichier D:\temp\otv\Donnees_source\CD16\B15_2018.xlsx on traite les permanents.<br> à partir des donnees sur PIGMA on traite les compteurs temporaires <br>***attention : il y a aussi des données de comptages temporaires en FIM qui permettront d'obtenir de la données horaires***

In [180]:
cd16=it.Comptage_cd16(r'D:\temp\otv\Donnees_source\CD16\B15_2018.xlsx',r'D:\temp\otv\Donnees_source\CD16\pigma\comptages_routiers_2019\comptages_routiers.shp',
                      r'D:\temp\otv\Donnees_source\CD16\pigma\position_compteurs_2019\position_compteurs.shp',2018)
cd16.comptage_forme()
cd16.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')

cd16.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')

cd16.update_bdd_16('local_otv','comptage','na_2010_2018_p')

## **CD 86**
- Année 2018 : 
> à partir des fichier D:\temp\otv\Donnees_source\CD86\comptages permanents 2018.xlsx et  D:\temp\otv\Donnees_source\CD86\Postes secondaires 2018.xls on traite les permanents et secondaires <br> il y a un petit pb sur les compteurs permanents entre la donnees pr+abs chez nous et la leur dans le tableau, donc il faut la premiere fois tout passer dans la table de correspondance

In [296]:
donnees_brutes_perm=pd.read_excel(r'D:\temp\otv\Donnees_source\CD86\comptages permanents 2018.xlsx')
donnees_brutes_second=pd.read_excel(r'D:\temp\otv\Donnees_source\CD86\Postes secondaires 2018.xls')

In [297]:
#mise en forme des données
donnees_brutes_perm['route']=donnees_brutes_perm['AXE'].apply(lambda x : Outils.epurationNomRoute(x[3:]))
donnees_brutes_perm['pr']=donnees_brutes_perm.apply(lambda x : int(x['CUMULD']//1000) if not pd.isnull(x['CUMULD']) else x['PLOD'], axis=1)
donnees_brutes_perm['absc']=donnees_brutes_perm.apply(lambda x : int(x['CUMULD']%1000) if not pd.isnull(x['CUMULD']) else 0, axis=1)
donnees_brutes_perm['id_gest']=donnees_brutes_perm.apply(lambda x : f"86-{x['route']}-{x['pr']}+{x['absc']}", axis=1)

In [298]:
cd86=it.Comptage_cd86(r'D:\temp\otv\Donnees_source\CD86\comptages permanents 2018.xlsx',r'D:\temp\otv\Donnees_source\CD86\Postes secondaires 2018.xls')

In [299]:
#traiter les perm : tous les PR et ABS des pt de comptages perm varient un peu, dc il faut un dico de corrsp
cd86.comptag_existant_bdd('local_otv', 'na_2010_2018_p', schema='comptage',dep='86')

#construire un dico de correspondance
corresp=cd86.existant[['id_comptag','route','pr','abs']].merge(donnees_brutes_perm, left_on=['route','pr'], right_on=['route','PLOD'], how='right').sort_values('id_comptag')
corresp['id_comptag']=corresp.apply(lambda x : x['id_comptag'] if not pd.isnull(x['id_comptag']) else f"86-{x['route']}-{x['pr_y']}+{x['absc']}", axis=1)
#le dico de correspondance à inserer dans corresp_id_comptage
corresp_id_comptag=corresp[['id_comptag', 'id_gest']].loc[corresp['id_comptag']!=corresp['id_gest']].rename(columns={'id_comptag':'id_gti'})
#la future df_attr, avec que les perm
df_cpt_perm=corresp[['id_comptag','TMJA','POURCENTAGE_PL','TYPE_POSTE','route','pr_y','absc']].rename(columns={'TMJA':'tmja','POURCENTAGE_PL':'pc_pl','TYPE_POSTE':'type_poste','pr_y':'pr'})
df_cpt_perm['type_poste']='permanent'

In [315]:
#traiter les comptages secondaires
donnees_brutes_second['route']=donnees_brutes_second.apply(lambda x : 'D'+str(x['RD']).strip(), axis=1)
donnees_brutes_second['pr']=donnees_brutes_second.apply(lambda x : re.split('(\.|\+)',str(x['PR']))[0], axis=1)
donnees_brutes_second['absc']=donnees_brutes_second.apply(lambda x : int(re.split('(\.|\+|,)',str(x['PR']))[2]+'0'*(3-len(re.split('(\.|\+|,)',str(x['PR']))[2]))) if re.search('(\.|\+)',str(x['PR'])) else 0, axis=1)
donnees_brutes_second['id_comptag']=donnees_brutes_second.apply(lambda x : f"86-{x['route']}-{x['pr']}+{x['absc']}", axis=1)
donnees_brutes_second['type_poste']='tournant'
df_cpt_second=donnees_brutes_second.rename(columns={'TV':'tmja','% PL':'pc_pl'}).drop(['LIEUX', 'RD','PR','n° de compteur','PL'], axis=1)

In [316]:
df_attr=pd.concat([df_cpt_perm,df_cpt_second], axis=0, sort=False)

In [317]:
df_attr_update=df_attr.loc[df_attr.id_comptag.isin(cd86.existant.id_comptag.tolist())].copy()
df_attr_insert=df_attr.loc[~df_attr.id_comptag.isin(cd86.existant.id_comptag.tolist())].copy()

In [380]:
cd86.df_attr_update.loc[cd86.df_attr_update['type_poste']=='tournant']

,id_comptag,tmja,pc_pl,type_poste,route,pr,absc,src
0,86-D8-32+50,940.0,0.077660,tournant,D8,32,50,tableur
1,86-D729-5+220,1640.0,0.076220,tournant,D729,5,220,tableur
2,86-D749-49+700,3370.0,0.047478,tournant,D749,49,700,tableur
3,86-D742-20+50,2090.0,0.047847,tournant,D742,20,50,tableur
4,86-D742-8+500,6960.0,0.023707,tournant,D742,8,500,tableur
5,86-D4-21+970,6150.0,0.021138,tournant,D4,21,970,tableur
6,86-D741-2+375,6795.0,0.033848,tournant,D741,2,375,tableur
7,86-D3-43+510,3850.0,0.020779,tournant,D3,43,510,tableur
8,86-D6-31+100,4686.0,0.051430,tournant,D6,31,100,tableur
9,86-D757-18+120,2660.0,0.020677,tournant,D757,18,120,tableur


In [355]:
cd86=it.Comptage_cd86(r'D:\temp\otv\Donnees_source\CD86\comptages permanents 2018.xlsx',r'D:\temp\otv\Donnees_source\CD86\Postes secondaires 2018.xls')

#si besoin de dico corresp : 
corr=cd86.corresp_perm('local_otv', 'na_2010_2018_p')
cd86.insert_bdd('local_otv', 'comptage','corresp_id_comptag',corr)

#sinon
cd86.comptage_forme()
cd86.classer_comptage_update_insert('local_otv', 'na_2010_2018_p')
cd86.update_bdd_86('local_otv','comptage','na_2010_2018_p')

In [376]:
cd86.df_attr_update

,id_comptag,tmja,pc_pl,type_poste,route,pr,absc,src
0,86-D148-0+10,3960.0,14.700000,permanent,D148,0,10,tableur
1,86-D148-22+410,4450.0,13.700000,permanent,D148,21,792,tableur
2,86-D148-32+560,4420.0,16.000000,permanent,D148,31,755,tableur
3,86-D749-22+0,5710.0,4.900000,permanent,D749,17,574,tableur
4,86-D749-41+0,4210.0,6.700000,permanent,D749,36,478,tableur
5,86-D749-61+940,2480.0,5.000000,permanent,D749,57,282,tableur
6,86-D725-9+980,3498.0,6.800000,permanent,D725,9,959,tableur
7,86-D1-83+130,1852.0,9.000000,permanent,D1,84,483,tableur
8,86-D759-17+400,3525.0,15.000000,permanent,D759,17,807,tableur
9,86-D759-6+310,3510.0,16.000000,permanent,D759,6,307,tableur


## **Ville Anglet**
> Ci-dessous simplement un exemple d'ouverture de fcihier .mdb, de liste des tables et de lecture d'une table

In [95]:
with ct.ConnexionBdd('mdb', fichierMdb=r'Q:\DAIT\TI\DREAL33\2020\OTV\Doc_travail\Donnees_source\Anglet_Commune\COMPTAGES ANGLET\Comptages 2019\donnees brutes\1.Av .des Dauphins.mdb') as c:
    tables = list(c.mdbCurs.tables())
    query = "SELECT * FROM A1"
    df = pd.read_sql(query, c.connexionMdb)

In [97]:
tables

[('Q:\\DAIT\\TI\\DREAL33\\2020\\OTV\\Doc_travail\\Donnees_source\\Anglet_Commune\\COMPTAGES ANGLET\\Comptages 2019\\donnees brutes\\1.Av .des Dauphins.mdb', None, 'MSysACEs', 'SYSTEM TABLE', None),
 ('Q:\\DAIT\\TI\\DREAL33\\2020\\OTV\\Doc_travail\\Donnees_source\\Anglet_Commune\\COMPTAGES ANGLET\\Comptages 2019\\donnees brutes\\1.Av .des Dauphins.mdb', None, 'MSysObjects', 'SYSTEM TABLE', None),
 ('Q:\\DAIT\\TI\\DREAL33\\2020\\OTV\\Doc_travail\\Donnees_source\\Anglet_Commune\\COMPTAGES ANGLET\\Comptages 2019\\donnees brutes\\1.Av .des Dauphins.mdb', None, 'MSysQueries', 'SYSTEM TABLE', None),
 ('Q:\\DAIT\\TI\\DREAL33\\2020\\OTV\\Doc_travail\\Donnees_source\\Anglet_Commune\\COMPTAGES ANGLET\\Comptages 2019\\donnees brutes\\1.Av .des Dauphins.mdb', None, 'MSysRelationships', 'SYSTEM TABLE', None),
 ('Q:\\DAIT\\TI\\DREAL33\\2020\\OTV\\Doc_travail\\Donnees_source\\Anglet_Commune\\COMPTAGES ANGLET\\Comptages 2019\\donnees brutes\\1.Av .des Dauphins.mdb', None, 'A1', 'TABLE', None),
 ('Q:\\D

In [98]:
df.head(1)

,ATime,AdviceCode,Speed,Length,StopTime,RoadTemp,Order,OCCFactor,GAP,Headway
0,2019-05-27 12:02:07,210,2450,7350,182,182,1,3.0,0,0


# ***TESTS***

In [ ]:
## DETAILS OPERATIONS CD87

#regrouper les différentes données issues de ficiers et les ajouter au dico
for k, v in d87.dico_voie.items() : 
    for i,e in enumerate(v) : 
        if len(e['fichiers'])==1 : 
            print(e['fichiers'][0])
            obj_fim=it.FIM(os.path.join(d87.dossier,e['fichiers'][0]))
            try : 
                obj_fim.resume_indicateurs()
            except obj_fim.fim_PasAssezMesureError : 
                continue
            except Exception as ex : 
                print(f"erreur : {ex} \n dans fichier : {e['fichiers'][0]}")
            e['tmja'], e['pc_pl'], e['date_debut'], e['date_fin']=obj_fim.tmja, obj_fim.pc_pl, obj_fim.date_debut,obj_fim.date_fin
        elif len(e['fichiers'])>1 :
            list_tmja=[]
            list_pc_pl=[]
            for f in e['fichiers'] : 
                obj_fim=it.FIM(os.path.join(d87.dossier,f))
                print(f)
                try : 
                    obj_fim.resume_indicateurs()
                except (obj_fim.fim_PasAssezMesureError,obj_fim.fimNbBlocDonneesError)  : 
                    continue
                except Exception as ex : 
                    print(f"erreur : {ex} \n dans fichier : {f}")
                list_tmja.append(obj_fim.tmja)
                list_pc_pl.append(obj_fim.pc_pl)
            e['tmja'], e['pc_pl'], e['date_debut'], e['date_fin']=int(mean(list_tmja)), round(mean(list_pc_pl),2),np.NaN, np.NaN

#renseigner le type de poste
for k, v in d87.dico_voie.items() : 
    for e in v : 
        if len(e['fichiers']) > 4 :
            e['type_poste']='permanent'
        elif 1<len(e['fichiers'])<=4 : 
            e['type_poste']='tournant'
        elif len(e['fichiers'])== 1 :
            e['type_poste']='ponctuel'
        else : 
            e['type_poste']='NC'

#faire une df avec les points de comptage
d87.df_attr=pd.DataFrame([[k, e['pr'], e['abs'], e['tmja'], e['pc_pl'], e['type_poste'],
                      e['date_debut'],e['date_fin']] for k, v in d87.dico_voie.items() for e in v if 'tmja' in e.keys()], 
             columns=['route','pr','absc','tmja','pc_pl','type_poste','date_debut','date_fin'])
d87.df_attr['id_comptag']=d87.df_attr.apply(lambda x :'87-'+x['route']+'-'+str(x['pr'])+'+'+str(x['absc']), axis=1)

#filtre cpt ponctuel
d87.df_attr=d87.df_attr.loc[d87.df_attr.apply(lambda x : x['date_debut'].month not in [7,8] and x['date_fin'].month not in [7,8], 
                                                         axis=1)].copy()

#fare le tri avec les comptages existants : 
#recuperer les compmtages existants
d87.comptag_existant_bdd('gti_otv_pg11', 'na_2010_2018_p', schema='comptage',dep='87', type_poste=False)
d87.df_attr_update=d87.df_attr.loc[d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
d87.df_attr_insert=d87.df_attr.loc[~d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
#obtenir une cle de correspondace pour les comptages tournants et permanents
df_correspondance=d87.corresp_old_new_comptag('gti_otv_pg11', 'public','d87_cpt_temp','lineaire.traf2017_bdt87_ed17_l',
                                'referentiel', 'troncon_route_bdt87_ed17_l','troncon_route_bdt87_ed17_l_vertices_pgr','id')

#passer la df de correspondance dans le table corresp_id_comptage
d87.insert_bdd('gti_otv_pg11', 'comptage', 'corresp_id_comptag', 
               df_correspondance.rename(columns={'id_comptag_lin':'id_gti','id_comptag':'id_gest'})[['id_gest','id_gti']])

#faire la correspondance entre les noms de comptage
d87.corresp_nom_id_comptag('gti_otv_pg11',d87.df_attr)

#recalculer les insert et update
d87.df_attr_update=d87.df_attr.loc[d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()
d87.df_attr_insert=d87.df_attr.loc[~d87.df_attr.id_comptag.isin(d87.existant.id_comptag.tolist())].copy()

#mettre en forme pour update
d87.df_attr_update['obs']=d87.df_attr_update.apply(lambda x : x['date_debut'].strftime('%d/%m/%Y')+'-'+ x['date_fin'].strftime('%d/%m/%Y') if not pd.isnull(x['date_debut']) else '', axis=1)
d87.df_attr_update.loc[d87.df_attr_update.pc_pl.isna(),'obs']='pc_pl inconnu'
d87.df_attr_update.loc[d87.df_attr_update.pc_pl.isna(),'pc_pl']=-99

#preparer update
valeurs_txt=d87.creer_valeur_txt_update(d87.df_attr_update, ['id_comptag','tmja','pc_pl','obs'])
dico_attr={'tmja_2018':'tmja','pc_pl_2018':'pc_pl','obs_2018':'obs'}
#update
d87.update_bdd('gti_otv_pg11', 'comptage', 'na_2010_2018_p', valeurs_txt,dico_attr)

#mettre en forme le insert
dbl=d87.df_attr_insert.loc[d87.df_attr_insert.duplicated('id_comptag', False)].copy()
ss_dbl=d87.df_attr_insert.loc[~d87.df_attr_insert.index.isin(dbl.index.tolist())].copy()
dbl=dbl.dropna()
dbl_traite=dbl.loc[dbl.tmja==dbl.groupby('id_comptag').tmja.transform(max)].drop_duplicates().copy()
d87.df_attr_insert=pd.concat([dbl_traite,ss_dbl], axis=0, sort=False)
d87.df_attr_insert.pc_pl.fillna(-99, inplace=True)
annee='2018'
d87.df_attr_insert['dep']='87'
d87.df_attr_insert['reseau']='RD'
d87.df_attr_insert['gestionnai']='CD87'
d87.df_attr_insert['concession']='N'
d87.df_attr_insert['obs']=d87.df_attr_insert.apply(lambda x : f"""nouveau_point,{x['date_debut'].strftime("%d/%m/%Y")}-{x['date_fin'].strftime("%d/%m/%Y")}""" if not (pd.isnull(x['date_debut']) and  pd.isnull(x['date_fin'])) else None,axis=1)
d87.df_attr_insert.rename(columns={'absc' : 'abs', 'tmja':'tmja_'+annee,'pc_pl':'pc_pl_'+annee,'obs':'obs_'+annee},inplace=True)
d87.df_attr_insert.drop(['date_debut','date_fin','route'],axis=1,inplace=True)

#mettre à jour la geom
d87.maj_geom('gti_otv_pg11', 'comptage', 'na_2010_2018_p', dep='87')